In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from time import sleep
import pandas as pd
import numpy as np
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import re
import copy
from bs4 import BeautifulSoup
import pickle
import glob
from datetime import timedelta, date
import urllib.request
from multiprocessing import Pool
import time
from http_request_randomizer.requests.proxy.requestProxy import RequestProxy
import random

req_proxy = RequestProxy() #you may get different number of proxy when  you run this at each time
proxies = req_proxy.get_proxy_list() #this will create proxy list

## Step 1: Get html links for all cases

In [2]:
# From 01/01/1900 to 09/10/2020: 8096951 cases
# From 01/01/1850 to 09/10/2020: 8100894 cases
# From 01/01/1800 to 09/10/2020: 8101326 cases
# From 01/01/1700 to 09/10/2020: 8101326 cases

In [3]:
total_page_number = int(8101326/20)+1

In [4]:
total_page_number

405067

In [5]:
all_url_list = ['https://www.leagle.com/leaglesearch?page='+str(i+1)+'&tod=09/10/2020&frd=01/01/1700' for i in range(total_page_number)]

In [6]:
all_url_list = ['https://www.leagle.com/leaglesearch?page='+str(i+1)+'&tod=09%2F10%2F2020&frd=01%2F01%2F1700' for i in range(total_page_number)]

In [ ]:
PROXY = proxies[random.randint(0,(len(proxies)-1))].get_address()
        webdriver.DesiredCapabilities.CHROME['proxy']={
            "httpProxy":PROXY,
            "ftpProxy":PROXY,
            "sslProxy":PROXY,

            "proxyType":"MANUAL",

        }

In [9]:
def get_html_page_source(url):
    try:
        
        chrome_options=Options()
        #chrome_options.add_argument('--headless')
        #br = webdriver.Chrome(options=chrome_options,executable_path=r'C:\Users\i26184\Desktop\chromedriver.exe')
        chrome_options.add_experimental_option("prefs", {
          "download.prompt_for_download": False,
          "download.directory_upgrade": True,
          "safebrowsing.enabled": False
        })
        br = webdriver.Chrome(options=chrome_options, executable_path='/home/ubuntu/chromedriver')
        br.get(url)

        element = WebDriverWait(br, 25).until(EC.presence_of_element_located((By.XPATH, "//div[@class='search-result-1']")))
        #page_number = url.replace('https://www.leagle.com/leaglesearch?page=','').replace('&tod=09/10/2020&frd=01/01/1700','')
        page_number = url.replace('https://www.leagle.com/leaglesearch?page=','').replace('&tod=09%2F10%2F2020&frd=01%2F01%2F1700','')
        file_name = '/home/ubuntu/leagle_all_cases/page_'+page_number+'.html'

        with open(file_name, "w") as file:
            file.write(br.page_source)
        br.close()
        return None
    
    except:
        br.close()
        return None

In [10]:
existing_page = glob.glob('/home/ubuntu/leagle_all_cases/*', recursive=True)

In [11]:
existing_page = [x.split('/')[-1].replace('.html','') for x in existing_page]

In [12]:
existing_page

['page_1']

In [13]:
all_url_page = [x.replace('https://www.leagle.com/leaglesearch?','').replace('&tod=09%2F10%2F2020&frd=01%2F01%2F1700','').replace('=','_') for x in all_url_list]

In [14]:
all_url_dict = dict(zip(all_url_page,all_url_list))

In [15]:
remain_page = set(all_url_page) - set(existing_page)

In [16]:
remain_url = [all_url_dict[x] for x in remain_page]

In [17]:
len(all_url_list)

405067

In [18]:
len(remain_url)

405066

In [ ]:
get_html_page_source(remain_url[0])

In [ ]:
remain_url[0]

In [ ]:
start_time = time.time()
p = Pool(8)
result = p.map(get_html_page_source,remain_url)
p.close()
p.join()
end_time = time.time()
print("--- %s seconds ---" % (end_time - start_time)) # 784